In [4]:
import pandas as pd
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')   # <-- add this
nltk.download('wordnet')

df = pd.read_csv("/content/train.csv")


stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return " ".join(tokens)

df["clean_text"] = df["text"].apply(clean_text)


tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=5000)
X = tfidf.fit_transform(df["clean_text"]).toarray()
y = df["target"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


mlp_model = models.Sequential([
    layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
mlp_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
mlp_model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.1)


tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df["clean_text"])
X_seq = tokenizer.texts_to_sequences(df["clean_text"])
X_pad = pad_sequences(X_seq, maxlen=50)

X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(
    X_pad, y, test_size=0.2, random_state=42
)


cnn_model = models.Sequential([
    layers.Embedding(input_dim=10000, output_dim=128, input_length=50),
    layers.Conv1D(128, 5, activation='relu'),
    layers.GlobalMaxPooling1D(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_seq, y_train_seq, epochs=5, batch_size=64, validation_split=0.1)


bilstm_model = models.Sequential([
    layers.Embedding(input_dim=10000, output_dim=128, input_length=50),
    layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
    layers.GlobalMaxPooling1D(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
bilstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
bilstm_model.fit(X_train_seq, y_train_seq, epochs=5, batch_size=64, validation_split=0.1)


def evaluate_model(model, X_test, y_test, deep=True):
    if deep:
        y_pred = (model.predict(X_test) > 0.5).astype("int32")
    else:
        y_pred = model.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("F1-score:", f1_score(y_test, y_pred))


print("\nMLP Results:")
evaluate_model(mlp_model, X_test, y_test, deep=True)

print("\nCNN Results:")
evaluate_model(cnn_model, X_test_seq, y_test_seq, deep=True)

print("\nBi-LSTM Results:")
evaluate_model(bilstm_model, X_test_seq, y_test_seq, deep=True)


log_reg = LogisticRegression(max_iter=200)
log_reg.fit(X_train, y_train)
print("\nLogistic Regression Results:")
evaluate_model(log_reg, X_test, y_test, deep=False)

svm = LinearSVC()
svm.fit(X_train, y_train)
print("\nSVM Results:")
evaluate_model(svm, X_test, y_test, deep=False)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.5828 - loss: 0.6611 - val_accuracy: 0.7931 - val_loss: 0.4542
Epoch 2/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8430 - loss: 0.3608 - val_accuracy: 0.7898 - val_loss: 0.4506
Epoch 3/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8964 - loss: 0.2639 - val_accuracy: 0.7882 - val_loss: 0.5032
Epoch 4/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9344 - loss: 0.1912 - val_accuracy: 0.7800 - val_loss: 0.5606
Epoch 5/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9484 - loss: 0.1543 - val_accuracy: 0.7783 - val_loss: 0.6640
Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


86/86 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.6226 - loss: 0.6466 - val_accuracy: 0.8112 - val_loss: 0.4329
Epoch 2/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.8787 - loss: 0.2971 - val_accuracy: 0.8062 - val_loss: 0.4382
Epoch 3/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.9464 - loss: 0.1645 - val_accuracy: 0.7931 - val_loss: 0.5121
Epoch 4/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.9723 - loss: 0.0903 - val_accuracy: 0.7603 - val_loss: 0.5981
Epoch 5/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.9751 - loss: 0.0817 - val_accuracy: 0.8046 - val_loss: 0.5804
Epoch 1/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - accuracy: 0.6109 - loss: 0.6501 - val_accuracy: 0.8177 - val_loss: 0.4258
Epoch 2/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 7s 76ms/step - accuracy: 0.8724 - loss: 0.3157 - val_accuracy: 0.7997 - val_loss: 0.4382
Epoch 3/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 10s 75ms/step - accuracy: 0.9323 - loss: 0.1930 - val_accuracy: 0.7833 - val_loss: 0.5206
